### House prices model

In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

- Load the houseprices data from Thinkful's database.
- Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
- Do you think your model is satisfactory? If so, why?
- In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.
- For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [34]:
# Load the houseprices data from Thinkful's database.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'  
postgres_pw = '7*.8G9QH21'  
postgres_host = '142.93.121.174'  
postgres_port = '5432'  
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

hp_df = pd.read_sql_query('select * from houseprices', con=engine)
engine.dispose()

hp_df.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [35]:
# Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?

hp_sel = hp_df[['saleprice', 'lotfrontage', 'lotarea', 'masvnrarea', 'bsmtfinsf1', 
                'totalbsmtsf', 'firstflrsf', 'secondflrsf', 'grlivarea', 'garagearea']]
hp_sel.replace(0, 1, inplace=True)
hp_sel.fillna(1, inplace=True)
hp_sellog = np.log(hp_sel)
hp_sellog_picked = hp_sellog[['saleprice', 'firstflrsf', 'grlivarea']]
print(hp_sellog_picked.head())

X = hp_sellog_picked[['firstflrsf', 'grlivarea']]
Y = hp_sellog_picked['saleprice']

X = sm.add_constant(X)
Y = hp_sellog_picked['saleprice']

results = sm.OLS(Y, X).fit()
print(results.summary())

   saleprice  firstflrsf  grlivarea
0  12.247694    6.752270   7.444249
1  12.109011    7.140453   7.140453
2  12.317167    6.824374   7.487734
3  11.849398    6.867974   7.448334
4  12.429216    7.043160   7.695303
                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.596
Model:                            OLS   Adj. R-squared:                  0.596
Method:                 Least Squares   F-statistic:                     1076.
Date:                Tue, 22 Oct 2019   Prob (F-statistic):          1.14e-287
Time:                        10:59:09   Log-Likelihood:                -69.293
No. Observations:                1460   AIC:                             144.6
Df Residuals:                    1457   BIC:                             160.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                    

Let's figure out some statistical values of my model rigidly selected by several verifications. 

1. F-test Value: It is '1.14e-287'. It is nearly zero. This means that the null hypothesis has been rejected by the significant of 1% and we could say that our model is useful and contributes something that is statistically significant in the explanation of the target.  
2. R-squared, adjusted R-squared: It has the same value, '0.596'. This value is not so low and not so high. This means that our model explains 59.6% of the variance in the charges, leaving 40.4% unexplained.
3. AIC and BIC: AIC is 144.6 and BIC is 160.4.

Every values could be judged with other models.  
So, let's check out other models.  

I am going to make two more models which have the same observation for the comparison.  

1. the model with 'hp_sellog'  
2. the 'hp_sellog_picked' model without log-scaled  

pd.set_option('display.max_rows', 2000)
hp_df_digits = hp_df.select_dtypes(exclude=['object'])
hp_df_digits.fillna(0)

hp_df_digits.isnull().sum()

In [37]:
# Let's make the first model
X1 = hp_sellog
Y1 = hp_sellog['saleprice']

X1 = sm.add_constant(X1)

results1 = sm.OLS(Y1, X1).fit()
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.282e+29
Date:                Tue, 22 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:00:47   Log-Likelihood:                 44984.
No. Observations:                1460   AIC:                        -8.995e+04
Df Residuals:                    1449   BIC:                        -8.989e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.332e-14   9.51e-15     -1.401      

In [39]:
# Let's make the second model

hp_sel_picked = hp_sel[['saleprice', 'firstflrsf', 'grlivarea']]

X2 = hp_sel_picked[['firstflrsf', 'grlivarea']]
Y2 = hp_sel_picked['saleprice']

X2 = sm.add_constant(X2)
results2 = sm.OLS(Y2, X2).fit()
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.564
Model:                            OLS   Adj. R-squared:                  0.563
Method:                 Least Squares   F-statistic:                     941.7
Date:                Tue, 22 Oct 2019   Prob (F-statistic):          3.05e-263
Time:                        11:03:49   Log-Likelihood:                -17938.
No. Observations:                1460   AIC:                         3.588e+04
Df Residuals:                    1457   BIC:                         3.590e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1.434e+04   4780.943     -3.000      0.0

In [50]:
data = [[1.14e-287, 0.596, 144.6, 160.4], [0.00, 1.000, -8.995e+04, -8.989e+04],
                [3.05e-263, 0.563, 3.588e+04, 3.590e+04]]
index_name = ['My model', 'Raw Data', 'Not Logged Selected Data']
col_name = ['F-test(prob)', 'Adj R-squared', 'AIC', 'BIC']
table = pd.DataFrame(data, index=index_name, columns=col_name)
table

,F-test(prob),Adj R-squared,AIC,BIC
My model,1.140000e-287,0.596,144.6,160.4
Raw Data,0.000000e+00,1.000,-89950.0,-89890.0
Not Logged Selected Data,3.050000e-263,0.563,35880.0,35900.0


Let's have a small talk about my three models.  

First one is my final model which I make for the result of former assignment. When we look into the details,  
- F-test probaility is quite low, almost near to zero. This means that this model's features could be enough to explain the outcome (target value) well in significant of 1%.  
- Adj R-sqaured is 0.596. This value is the second large value in my models. Also this means that 'My model' explains 59.6% of the variance in the target variable, leaving 40.4% unexplained.
- AIC & BIC is 144.6 and BIC is 160.4 this is the lowest value among the models.

Second one looks quite strange.  
- Adj R-sqaured is 1. It is not impossible. This means that this model is overfitting. 

Last model looks great in some way, but the overall judgement on it is not better than 'My model'. In this reason, the log-scaled model represent the outcome better than not log-scaled one.